# Johnston et al. (2024) - Secondary thalamic dysfunction underlies abnormal large-scale neural dynamics in chronic stroke

This notebook contains Python and R code used to generate the statistical results in Johnston et al. (2024).

# Setup

### Python setup:

In [1]:
#!pip install -r requirements.txt # Uncomment to install python dependencies


In [4]:
import scipy
from scipy.stats import t
import numpy as np
import pandas as pd
import math
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
from scipy.stats import zscore

# Load master_df_model (contains both spectral param info as well as 1-40 Hz full CT model sim parameters)
region_df = pd.read_csv('data/MEG_params_by_region.csv')

# Load thalamus measures (volume, MD, CBF) and degeneration score
thal_df = pd.read_csv('data/thalamus_quant.csv')

# Subcortical ROIs (to be excluded)
subcorts = ['Hippocampus_L', 'Hippocampus_R', 'Amygdala_L', 'Amygdala_R', 'Caudate_L', 'Caudate_R', 'Putamen_L', 'Putamen_R', 'Pallidum_L', 'Pallidum_R', 'Thalamus_L', 'Thalamus_R']

patients = ['P' + str(n) for n in np.arange(1,19)]
controls = ['P' + str(n) for n in np.arange(1,24)]


### R setup:

In [3]:
# Enable R 
os.environ['R_HOME'] = "C:/Users/pjohnston/AppData/Local/Programs/R/R-4.3.1/" # Path to your R installation with the packages below installed
%load_ext rpy2.ipython

C:\Users\pjohnston\Anaconda3\envs\stroke_eeg2\lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [4]:
%%R

library('lme4')
library('tidyverse')
library('lmeresampler')
library('lmerTest')
library('mediation')
library('dplyr')

-- Attaching core tidyverse packages ---------------------------------------------------------------- tidyverse 2.0.0 --
v dplyr     1.1.2     v readr     2.1.4
v forcats   1.0.0     v stringr   1.5.0
v ggplot2   3.4.2     v tibble    3.2.1
v lubridate 1.9.2     v tidyr     1.3.0
v purrr     1.0.1     
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x tidyr::expand() masks Matrix::expand()
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
x tidyr::pack()   masks Matrix::pack()
x tidyr::unpack() masks Matrix::unpack()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: Matrix

Attaching package: 'lmerTest'

The following object is masked from 'package:lme4':

    lmer

The following object is masked from 'package:stats':

    step

Loading required package: MASS

Attaching package: 'MASS'

The following object is masked from 'package:dplyr':

    select

Loading required package: mvtnorm
Loading required package: sandwich
mediation: Causal Mediation Analysis
Version: 4.5.0



### Define helper functions:

In [11]:
def normalize_column_by_controls(df, column_name):
    
    mean = np.mean(df[column_name][df.group=='control'])
    std = np.std(df[column_name][df.group=='control'])
    
    normalized_values = (df[column_name] - mean) / std
    
    return normalized_values

def compute_pca(df):
    # Impute missing values, standardize columns, compute PCA. Returns PCA and computed scores.
    
    # Perform imputation
    imputer = SimpleImputer(strategy='mean')
    df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
    
    # Perform column normalization
    scaler = StandardScaler()
    df_normalized = pd.DataFrame(scaler.fit_transform(df_imputed), columns=df_imputed.columns)
    
    # Perform PCA
    pca = PCA()
    pca.fit(df_normalized)
    
    scores = pca.transform(df_normalized)
    
    return pca, scores

def mean_and_sd(data, round_digits):
    mean = np.round(np.mean(data), round_digits) 
    sd = np.round(np.std(data), round_digits)
    
    return mean, sd

def cohen_d(d1, d2):
    # Compute Cohen's d 

     # calculate the size of samples
     n1, n2 = len(d1), len(d2)
     # calculate the variance of the samples
     s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
     # calculate the pooled standard deviation
     s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
     # calculate the means of the samples
     u1, u2 = np.mean(d1), np.mean(d2)
     # calculate the effect size
     return (u1 - u2) / s

def CI_ind(d1, d2, percent):
    # Calculate the given confidence interval for two independent samples

    outer_margin=(1-percent)/2
    
    N1 = len(d1)
    N2 = len(d2)
    df = (N1 + N2 - 2)
    std1 = np.std(d1)
    std2 = np.std(d2)
    std_N1N2 = np.sqrt( ((N1 - 1)*(std1)**2 + (N2 - 1)*(std2)**2) / df) 

    diff_mean = np.mean(d1) - np.mean(d2)
    MoE = t.ppf(1-outer_margin, df) * std_N1N2 * np.sqrt(1/N1 + 1/N2)
    
    return [diff_mean - MoE, diff_mean + MoE]

    
def pearsonr_CI(r, n):
    # Compute the 95% confidence interval for a Pearson correlation coefficient.

    # Fisher's z transformation
    z = np.arctanh(r)
    
    # Standard error of Fisher's z
    se = 1 / np.sqrt(n - 3)
    
    # Margin of error at 95% confidence level (z-score of 1.96)
    margin_error = 1.96 * se
    
    # Compute confidence interval for Fisher's z
    lower_z = z - margin_error
    upper_z = z + margin_error
    
    # Transform back to Pearson correlation scale
    lower_r = np.tanh(lower_z)
    upper_r = np.tanh(upper_z)
    
    return lower_r, upper_r


def report_group_diff(d1, d2, labels=[]):
    print(labels[0] + ": " + str(mean_and_sd(d1, 3)))
    print(labels[1] + ": " + str(mean_and_sd(d2, 3)))
    print(scipy.stats.ttest_ind(d1,d2))
    print("Cohen's d: " + str(cohen_d(d1, d2)))
    print('95% CI: ' + str(CI_ind(d1, d2, 0.95)))
    print('\n')

def compute_GLMs(df):
    # Compute GLM between each pair of columns in given df

    # Loop through each pair of columns
    for i, col1 in enumerate(df.columns):
        for j, col2 in enumerate(df.columns):
            if i < j:  # Ensure only unique pairs are considered
                d1 = df[col1]
                d2 = df[col2]

                # Remove nans from input
                nans = np.logical_or(np.isnan(d1), np.isnan(d2))
                d1 = d1[~nans]
                d2 = d2[~nans]

                # Standardize input
                d1 = [(x-np.mean(d1))/np.std(d1) for x in d1]
                d2 = [(x-np.mean(d2))/np.std(d2) for x in d2]

                # Fit linear model
                glm_fit = sm.GLM(d1, d2).fit()
                slope = glm_fit.params[0]
                p_value = glm_fit.pvalues[0]

                # Compute CI
                l_ci, u_ci = pearsonr_CI(slope, len(d1))

                print(col1 + ' ~ ' + col2)
                print('beta = ' + str(np.round(slope,3)))
                print('p = ' + str(np.round(p_value,3)))
                print('95% CI [' + str(np.round(l_ci, 4)) + ',' + str(np.round(u_ci, 4)) + ']')
                #print(scipy.stats.pearsonr(d1,d2))
                print('\n')
                
                glm_fit.summary()

                
def residualize(x, y, z):
    # Returns the residuals of x and y after regressing on z. Removes any row with nans from all three variables.
    
    # Create a mask to filter out NaN values in either X or Y
    mask = ~np.isnan(x) & ~np.isnan(y) & ~np.isnan(z)

    # Apply the mask to X, Y, and Z
    x = x[mask]
    y = y[mask]
    z = z[mask]

    data = pd.DataFrame({'x': x, 'y': y, 'z' : z})

    # Regression of X on Z
    model_x = smf.ols(formula='x~z', data=data)
    results_x = model_x.fit()
    residuals_x = results_x.resid

    # Regression of Y on Z
    model_y = smf.ols(formula='y~z', data=data)
    results_y = model_y.fit()
    residuals_y = results_y.resid
    
    return residuals_x, residuals_y


### Prepare subject-wise dataframe
Compute average left hemisphere, right hemisphere, and difference (left minus right) values for parameters of interest, add to dataframe.

Only average over regions with spheres that are >=50% intact.

In [14]:
# Parameters of interest
params = ['z', 'x', 'y', 'Gee', 'Gei', 'Gese', 'Gesre', 'Gsrs', 'alpha', 'beta', 't0', 'EMG', 'chisq', 'exp', 'ta_cf', 'ta_pw', 'ta_bw', 'b_cf', 'b_pw', 'b_bw', 'offset']

subj_df = thal_df # Create new subj-wise df

for param in params:

    lvals = []
    rvals = []
    diff_vals = []

    for subj in subj_df.subj:

        # Take only cortical ROIS with spheres >=50% intact
        trim_df = region_df[(region_df.subj == subj) & (~region_df.roi_label.isin(subcorts)) & (region_df.percent_sphere_in_lesion < 0.5)] 
        
        lmean = np.mean(trim_df[param][trim_df.hemi == 'L'])
        rmean = np.mean(trim_df[param][trim_df.hemi == 'R'])
        
        lvals.append(lmean)
        rvals.append(rmean)
        diff_vals.append(lmean-rmean)

    
    # Add lists of vals to dataframe
    subj_df['lcort_' + param] = lvals
    subj_df['rcort_' + param] = rvals
    subj_df['diff_' + param] = diff_vals

# Normalize left and right hemisphere values based on controls ('cnorm')
new_columns = []
for param in params:
    lcort_col = 'lcort_' + param + '_cnorm'
    rcort_col = 'rcort_' + param + '_cnorm'
    
    lcort_norm = normalize_column_by_controls(subj_df, 'lcort_' + param)
    rcort_norm = normalize_column_by_controls(subj_df, 'rcort_' + param)
    
    new_columns.extend([(lcort_col, lcort_norm), (rcort_col, rcort_norm)])

subj_df = pd.concat([subj_df, pd.DataFrame.from_dict(dict(new_columns))], axis=1)

# Results

## 2.1 Corticothalamic model fitting

Compare model goodness of fit between patients and controls:

In [15]:
d1 = subj_df.lcort_chisq[(subj_df.group == 'patient')]
d2 = subj_df.lcort_chisq[(subj_df.group == 'control')]
print("Chi square, L hemi average (mean, sd):")
report_group_diff(d1,d2, labels=['Patients', 'Controls'])


d1 = subj_df.rcort_chisq[(subj_df.group == 'patient')]
d2 = subj_df.rcort_chisq[(subj_df.group == 'control')]
print("Chi square, R hemi average (mean, sd):")
report_group_diff(d1,d2, labels=['Patients', 'Controls'])


# Calculate likelihood estimate as BrainTrak does and add to subj_df
subj_df['lcort_chisq_lk'] = [math.exp(-chisq/2) for chisq in subj_df['lcort_chisq']]
subj_df['rcort_chisq_lk'] = [math.exp(-chisq/2) for chisq in subj_df['rcort_chisq']]


print("Likelihood, whole brain (mean, sd):")
d1 = np.concatenate([subj_df.lcort_chisq_lk[subj_df.group == 'patient'], 
                     subj_df.rcort_chisq_lk[subj_df.group == 'patient']])
d2 = np.concatenate([subj_df.lcort_chisq_lk[subj_df.group == 'control'], 
                     subj_df.rcort_chisq_lk[subj_df.group == 'control']])           
print("Patient both hemis: " + str(mean_and_sd(d1, 3)))
print("Control both hemis: " + str(mean_and_sd(d2, 3)))
print(scipy.stats.ttest_ind(d1, d2))
print('\n')

# Left hemisphere patients vs controls
d1=subj_df.lcort_chisq_lk[(subj_df.group == 'patient')]
d2=subj_df.lcort_chisq_lk[(subj_df.group == 'control')]
print("Likelihood, L hemi (mean, sd):")
report_group_diff(d1,d2, labels=['Patients', 'Controls'])

# Right hemisphere patients vs controls
d1=subj_df.rcort_chisq_lk[(subj_df.group == 'patient')]
d2=subj_df.rcort_chisq_lk[(subj_df.group == 'control')]
print("Likelihood, R hemi (mean, sd):")
report_group_diff(d1,d2, labels=['Patients', 'Controls'])

Chi square, L hemi average (mean, sd):
Patients: (0.546, 0.157)
Controls: (0.664, 0.191)
Ttest_indResult(statistic=-2.0788620322816382, pvalue=0.044254774643103616)
Cohen's d: -0.654210372064878
95% CI: [-0.23172798513529386, -0.00600065619735729]


Chi square, R hemi average (mean, sd):
Patients: (0.61, 0.201)
Controls: (0.632, 0.177)
Ttest_indResult(statistic=-0.3763048182107243, pvalue=0.7087297753604778)
Cohen's d: -0.11842176696124869
95% CI: [-0.1425079452086266, 0.09683453004883466]


Likelihood, whole brain (mean, sd):
Patient both hemis: (0.752, 0.066)
Control both hemis: (0.726, 0.066)
Ttest_indResult(statistic=1.7588907325782865, pvalue=0.0824198773834603)


Likelihood, L hemi (mean, sd):
Patients: (0.764, 0.058)
Controls: (0.721, 0.067)
Ttest_indResult(statistic=2.112306188514975, pvalue=0.04111613367338227)
Cohen's d: 0.6647351272208512
95% CI: [0.0028221154541105548, 0.08315287752891191]


Likelihood, R hemi (mean, sd):
Patients: (0.741, 0.071)
Controls: (0.732, 0.064)
Tt

## 2.2 Stroke patients demonstrate decreased intrathalamic inhibition in the ipsilesional hemisphere

Examine group differences in the hemisphere-averaged model parameters:

In [16]:
# Choose hemisphere (lcort or rcort)
hemi = 'lcort' 

params_to_test = ['x', 'y', 'z', 'Gee', 'Gei', 'Gese', 'Gesre', 'Gsrs', 'alpha', 'beta', 't0', 'EMG']

for p in params_to_test:
    
    d1=subj_df[hemi + "_" + p][(subj_df.group == 'patient')]
    d2=subj_df[hemi + "_" + p][(subj_df.group == 'control')]
    print(p + " " +  hemi + " (mean, sd):")
    report_group_diff(d1, d2, labels=['Patients', 'Controls'])
    

x lcort (mean, sd):
Patients: (0.617, 0.055)
Controls: (0.592, 0.064)
Ttest_indResult(statistic=1.2569800097405508, pvalue=0.2162369970324673)
Cohen's d: 0.3955670684638644
95% CI: [-0.013905829347930297, 0.06269045019238653]


y lcort (mean, sd):
Patients: (0.106, 0.035)
Controls: (0.1, 0.031)
Ttest_indResult(statistic=0.5905480338380207, pvalue=0.5582301500713693)
Cohen's d: 0.18584333300624467
95% CI: [-0.014592277968964808, 0.027067611687784206]


z lcort (mean, sd):
Patients: (0.137, 0.034)
Controls: (0.214, 0.048)
Ttest_indResult(statistic=-5.65620767540552, pvalue=1.5548623135354315e-06)
Cohen's d: -1.7799881234744435
95% CI: [-0.10377385041097131, -0.050064642283882466]


Gee lcort (mean, sd):
Patients: (12.768, 0.752)
Controls: (12.812, 0.855)
Ttest_indResult(statistic=-0.1670280102887737, pvalue=0.8682110507049614)
Cohen's d: -0.052563111480921514
95% CI: [-0.5601777595175395, 0.47275567599114465]


Gei lcort (mean, sd):
Patients: (-19.996, 1.409)
Controls: (-20.85, 1.411)
Tt

In [17]:
### add Z and Gsrs correlation
d1=subj_df[hemi + "_z"][(subj_df.group == 'patient')]
d2=subj_df[hemi + "_Gsrs"][(subj_df.group == 'patient')]
r, p = scipy.stats.pearsonr(d1, d2)
l_ci, u_ci = pearsonr_CI(r, len(d1))
print('Pearson r = ', str(r))
print('p = ', str(p))
print('95% CI: [' + str(np.round(l_ci, 4)) + ',' + str(np.round(u_ci, 4)) + ']')

Pearson r =  -0.9017319975972135
p =  3.199748647685364e-07
95% CI: [-0.9631,-0.751]


## 2.3 Stroke patients show evidence of ipsilesional thalamic degeneration on multiple measures

Independent samples t-tests (patients vs controls):

In [18]:
# Choose thalamus (lthal or rthal)
hemi = 'lthal' 

thal_params = [hemi+'_volume_cnorm', hemi+'_MD_cnorm', hemi+'_CBF_cnorm']

for p in thal_params:
    
    d1=subj_df[p][(subj_df.group == 'patient')]
    d1=d1.dropna()
    d2=subj_df[p][(subj_df.group == 'control')]
    d2=d2.dropna()
    print(p + " " +  hemi + " (mean, sd):")
    
    report_group_diff(d1, d2, labels=['Patients', 'Controls'])
    

lthal_volume_cnorm lthal (mean, sd):
Patients: (-2.091, 1.587)
Controls: (0.0, 1.0)
Ttest_indResult(statistic=-5.019447770574209, pvalue=1.1778438088713163e-05)
Cohen's d: -1.579602081598216
95% CI: [-2.9112639246504486, -1.2702070062770874]


lthal_MD_cnorm lthal (mean, sd):
Patients: (2.022, 2.076)
Controls: (-0.0, 1.0)
Ttest_indResult(statistic=3.9717300821884036, pvalue=0.0003070706812700775)
Cohen's d: 1.2703440074501398
95% CI: [1.020025040170334, 3.02324026321074]


lthal_CBF_cnorm lthal (mean, sd):
Patients: (-0.442, 1.875)
Controls: (-0.0, 1.0)
Ttest_indResult(statistic=-0.9269864480766222, pvalue=0.35994118707402456)
Cohen's d: -0.3017739720755545
95% CI: [-1.3793348846528182, 0.49588108432673106]




Linear regression between thalamus measures (Figure 3):

In [19]:
glm_df = subj_df.loc[subj_df.group == 'patient', [hemi+'_volume_cnorm', hemi+'_MD_cnorm', hemi+'_CBF_cnorm']]

compute_GLMs(glm_df)

lthal_volume_cnorm ~ lthal_MD_cnorm
beta = -0.329
p = 0.163
95% CI [-0.6992,0.1799]


lthal_volume_cnorm ~ lthal_CBF_cnorm
beta = 0.404
p = 0.087
95% CI [-0.1151,0.7494]


lthal_MD_cnorm ~ lthal_CBF_cnorm
beta = -0.573
p = 0.007
95% CI [-0.8322,-0.1076]




Compute PCA on thalamus measures:

In [20]:
# Compute PCA with imputation and standardization
pca, scores = compute_pca(subj_df.loc[subj_df.group == 'patient', [hemi+'_volume_cnorm', hemi+'_MD_cnorm', hemi+'_CBF_cnorm']])

# Get components and invert so that higher score on PC1 means more degeneration 
components = pca.components_
components_inv = components*-1

# Print PCs and variance explained
print('PC loadings [volume, MD, CBF]')
print(components_inv)
print('\n')

print('Explained variance ratio (PC1, PC2, PC3)')
print(pca.explained_variance_ratio_)

PC loadings [volume, MD, CBF]
[[-0.50865915  0.59565035 -0.62166433]
 [-0.84798498 -0.47152384  0.24204699]
 [ 0.14895418 -0.65028143 -0.74494746]]


Explained variance ratio (PC1, PC2, PC3)
[0.62602351 0.23398508 0.13999141]


## 2.4 Model-estimated intrathalamic inhibition links thalamus degeneration and abnormal MEG dynamics

First compute MEG slowing score based on parameters identified in Johnston et al., 2023:

In [21]:
# Compute PCA
slowing_measures = ['lcort_exp_cnorm', 'lcort_ta_cf_cnorm', 'lcort_b_pw_cnorm']
slowing_pca_df = subj_df.loc[subj_df.group == 'patient', slowing_measures]
slowing_pca, slowing_pca_scores = compute_pca(slowing_pca_df)


# Add scores to small dataframe for merging by subj number to subj_df
lcort_slowing_scores_df = pd.DataFrame(data = {'subj' : subj_df.subj[subj_df.group == 'patient'], 
                                 'slowing_score_left' : slowing_pca_scores[:,0]})

# Merge with subject-wise dataframe (if column doesn't already exist)
if 'slowing_score_left' not in subj_df.keys():

    subj_df = pd.merge(subj_df, lcort_slowing_scores_df, how='left', left_on='subj', right_on='subj')


Compute linear regression on each pair of: thalamus degeneration score, Z, and MEG slowing score, and lesion volume:

In [22]:
glm_df2 = subj_df.loc[subj_df.group == 'patient', ['degen_score_left', 'lcort_z_cnorm', 'slowing_score_left', 'lesion_volume']]

compute_GLMs(glm_df2)

degen_score_left ~ lcort_z_cnorm
beta = -0.551
p = 0.006
95% CI [-0.8097,-0.1136]


degen_score_left ~ slowing_score_left
beta = 0.456
p = 0.035
95% CI [-0.0144,0.7606]


degen_score_left ~ lesion_volume
beta = 0.084
p = 0.729
95% CI [-0.3988,0.5298]


lcort_z_cnorm ~ slowing_score_left
beta = -0.871
p = 0.0
95% CI [-0.9511,-0.6809]


lcort_z_cnorm ~ lesion_volume
beta = 0.163
p = 0.495
95% CI [-0.3285,0.5856]


slowing_score_left ~ lesion_volume
beta = -0.087
p = 0.719
95% CI [-0.5321,0.3961]




Compute PCA on thalamus degeneration score, Z, and MEG slowing score:

In [23]:
# PCA
# Compute PCA with imputation and standardization
pca, scores = compute_pca(subj_df.loc[subj_df.group == 'patient', ['degen_score_left', 'lcort_z_cnorm', 'slowing_score_left']])

# Get components 
components = pca.components_

# Print PCs and variance explained
print('PC loadings [volume, MD, CBF]')
print(components)
print('\n')

print('Explained variance ratio (PC1, PC2, PC3)')
print(pca.explained_variance_ratio_)

PC loadings [volume, MD, CBF]
[[ 0.48938008 -0.62748593  0.60561418]
 [-0.86458625 -0.25831671  0.43100244]
 [ 0.1140077   0.7345297   0.66893076]]


Explained variance ratio (PC1, PC2, PC3)
[0.75682618 0.20274444 0.04042938]


In [16]:
# Save
#subj_df.to_csv('subj_df.csv')

Use Causal Mediation Analysis in R to determine whether Z mediates relationship between thalamus degeneration and MEG slowing.

Note: p values and confidence intervals will vary due to bootstrapping.

In [17]:
 # Pass patient data to R

patient_df = subj_df[subj_df.group == 'patient']

%R -i patient_df


In [18]:
%%R 

patient_df_scaled = as.data.frame(scale(patient_df[c('degen_score_left', 'lcort_z_cnorm', 'slowing_score_left')]))

model.M <- lm(lcort_z_cnorm ~ degen_score_left, patient_df_scaled) #M ~ X
model.Y <- lm(slowing_score_left ~ degen_score_left + lcort_z_cnorm, patient_df_scaled) #Y ~ M + X
med_results <- mediate(model.M, model.Y, treat='degen_score_left', mediator='lcort_z_cnorm',
                   boot=TRUE, sims=1000)


summary(med_results)


Causal Mediation Analysis 

Nonparametric Bootstrap Confidence Intervals with the Percentile Method

               Estimate 95% CI Lower 95% CI Upper p-value   
ACME             0.4908       0.1955         0.91   0.002 **
ADE             -0.0353      -0.3491         0.19   0.632   
Total Effect     0.4555       0.0756         0.86   0.022 * 
Prop. Mediated   1.0775       0.7142         2.76   0.020 * 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Sample Size Used: 18 


Simulations: 1000 



Running nonparametric bootstrap



Sensitivity analysis:

In [19]:
%%R

sens = medsens(med_results,
               rho.by=0.1,
               effect.type='indirect')

summary(sens)


Mediation Sensitivity Analysis for Average Causal Mediation Effect

Sensitivity Region

      Rho    ACME 95% CI Lower 95% CI Upper R^2_M*R^2_Y* R^2_M~R^2_Y~
[1,] -0.9 -0.1781      -0.3784       0.0223         0.81       0.1357
[2,] -0.8  0.0588      -0.1009       0.2185         0.64       0.1072
[3,] -0.7  0.1732      -0.0249       0.3713         0.49       0.0821

Rho at which ACME = 0: -0.8
R^2_M*R^2_Y* at which ACME = 0: 0.64
R^2_M~R^2_Y~ at which ACME = 0: 0.1072 



## 2.5 Lesion proximity predicts MEG spectral slowing

Use a linear mixed effects model in R to explore the effect of proximity to lesion on spectral slowing:

In [24]:
# Load df with ipsilesional spectra/model parameters normalized by controls for each region
roi_df_left = pd.read_csv('data/roi_df_left.csv')

# Pass region-level data to R
%R -i roi_df_left

UsageError: Line magic function `%R` not found.


In [21]:
%%R

# Scale numeric columns only 
roi_df_left_scaled = roi_df_left
roi_df_left_scaled = roi_df_left_scaled %>% mutate(across(where(is.numeric), scale))

# Create a normalized "proximity to lesion" measure by multiplying Z-scored distance by *-1
roi_df_left_scaled$prox_to_lesion = -1*roi_df_left_scaled$dist_to_lesion

# Mixed effects model with effect of left thalamus degen score (control normalized), distance to lesion, and their interaction (plus lesion size) with varying intercept and slope for each patient
#slowing_model <- lmer(roi_slowing_pc_score ~ degen_score_excl3_left * prox_to_lesion + lesion_volume + (1|subj), data=roi_df_left_scaled)
slowing_model <- lmer(roi_slowing_pc_score ~ degen_score_left * prox_to_lesion + lesion_volume + (prox_to_lesion|subj), data=roi_df_left_scaled)
# Bootstrap slowing model
slowing_model_boot <- bootstrap(slowing_model, .f=fixef, type='parametric', B=1000) # Only need to estimate the fixed effects for now

summary(slowing_model)


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
roi_slowing_pc_score ~ degen_score_left * prox_to_lesion + lesion_volume +  
    (prox_to_lesion | subj)
   Data: roi_df_left_scaled

REML criterion at convergence: 1007.7

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.7057 -0.5971  0.0333  0.5456  4.0057 

Random effects:
 Groups   Name           Variance Std.Dev. Corr 
 subj     (Intercept)    0.39228  0.6263        
          prox_to_lesion 0.06609  0.2571   -0.09
 Residual                0.46325  0.6806        
Number of obs: 447, groups:  subj, 18

Fixed effects:
                                Estimate Std. Error       df t value Pr(>|t|)
(Intercept)                      0.04214    0.15491 15.14463   0.272 0.789259
degen_score_left                 0.26651    0.14741 15.29084   1.808 0.090335
prox_to_lesion                   0.36245    0.07647 15.72656   4.740 0.000232
lesion_volume                   -0.16171    0.14373

## 2.6 Thalamic degeneration is correlated with cognitive and language impairment

Compute Spearman correlation between thalamus degeneration and cognitive/language measures after controlling for effect of lesion size.

In [25]:
# Load behavioural data
behav_df = pd.read_csv('data/behav.csv')

# Create new dataframe merging subj_df and behavioural measures
corr_df = subj_df.loc[subj_df.group == 'patient', ['subj', 'degen_score_left', 'lcort_z', 'slowing_score_left', 'lesion_volume']].merge(behav_df, on='subj', how='left')


In [26]:
behavs = ['moca', 'bnt', 'ppvt', 'wab_aphasia_score']

# Compute Spearman correlations between thalamus degen and behavioural measures, after residualizing by lesion volume
x = corr_df['degen_score_left']
z = corr_df['lesion_volume']

for b in behavs:

    y = corr_df[b]
    x_resid, y_resid = residualize(x,y,z) # Residualize on lesion volume

    print(b + ' ~ thalamus degeneration')
    r, p = scipy.stats.spearmanr(x_resid, y_resid)
    print(f'Spearman r: {r}, p = {p}')
    print('\n')
    
    #plt.scatter(x_resid, y_resid)
    #plt.show()

moca ~ thalamus degeneration
Spearman r: -0.6868131868131868, p = 0.009509496050393583


bnt ~ thalamus degeneration
Spearman r: -0.5612745098039216, p = 0.019063465403200246


ppvt ~ thalamus degeneration
Spearman r: -0.556372549019608, p = 0.02037058662451899


wab_aphasia_score ~ thalamus degeneration
Spearman r: -0.2916666666666667, p = 0.2560026072263




Check correlations between lesion volume and raw behavioural measures

In [31]:
for b in behavs:

    x = corr_df['lesion_volume']
    y = corr_df[b]
    
    # Remove nans from input
    nans = np.logical_or(np.isnan(x), np.isnan(y))
    x = x[~nans]
    y = y[~nans]
    
    print(b + ' ~ lesion_volume')
    r, p = scipy.stats.spearmanr(x, y)
    print(f'Spearman r: {r}, p = {p}')
    print('\n')
    
    #plt.scatter(x_resid, y_resid)
    #plt.show()

moca ~ lesion_volume
Spearman r: -0.23933997882811733, p = 0.43095955006186404


bnt ~ lesion_volume
Spearman r: -0.2015149839839546, p = 0.4379920364155855


ppvt ~ lesion_volume
Spearman r: -0.31592036089432557, p = 0.21671971676993526


wab_aphasia_score ~ lesion_volume
Spearman r: -0.4757818184278815, p = 0.05355540911569535




# Supplementary Material

## S1 Thalamus degeneration and primary tissue damage differentially predict language impairment. 

In [8]:
# Load patient info for regression
reg_df = pd.read_csv('data/reg_df.csv')

y = reg_df['bnt'].copy() # 'bnt', 'ppvt', or 'wab_aphasia_score'

predictors = ['degen_score_excl3_left', 'Angular_L', 'Temporal_Sup_L', 'IFG_L']
#['degen_score_excl3_left', 'lesion_volume'] + lang_regions
X = reg_df[predictors].copy()


# Normalize
X = zscore(X, nan_policy='omit')
y = zscore(y, nan_policy='omit')

X2 = sm.add_constant(X)
est = sm.OLS(y, X2, missing='drop')
est2 = est.fit()
print(est2.summary())


                            OLS Regression Results                            
Dep. Variable:                    bnt   R-squared:                       0.429
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                     2.250
Date:                Fri, 19 Jul 2024   Prob (F-statistic):              0.124
Time:                        10:29:56   Log-Likelihood:                -19.365
No. Observations:                  17   AIC:                             48.73
Df Residuals:                      12   BIC:                             52.90
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -0